In [ ]:
import base64
import hmac
import os
import struct
import time
from urllib.parse import urlparse, parse_qs

import requests
from fyers_apiv3 import fyersModel

totp_key = "xxxxxxxxxxxxxx"  # totp_key (ex., "OMKRABCDCDVDFGECLWXK6OVB7T4DTKU5")
username = "xxxxxxxxxxxxxx"  # Fyers Client ID (ex., "TK01248")
pin = 0000  # four-digit PIN
client_id = "xxxxxxxxxxxxxx"  # App ID of the created app (ex., "L9NY305RTW-100")
secret_key = "xxxxxxxxxxxxxx"  # Secret ID of the created app
redirect_uri = "xxxxxxxxxxxxxx"  # Redirect URL you entered while creating the app (ex., "https://trade.fyers.in/api-login/redirect-uri/index.html")



def read_file():
    try:
        with open("fyers_token.txt", "r") as f:
            token = f.read().strip()
        return token
    except FileNotFoundError:
        return None


def write_file(token):
    with open("access_token.txt", "w") as f:
        f.write(token)


def totp(key, time_step=30, digits=6, digest="sha1"):
    key = base64.b32decode(key.upper() + "=" * ((8 - len(key)) % 8))
    counter = struct.pack(">Q", int(time.time() / time_step))
    mac = hmac.new(key, counter, digest).digest()
    offset = mac[-1] & 0x0F
    binary = struct.unpack(">L", mac[offset : offset + 4])[0] & 0x7FFFFFFF
    return str(binary)[-digits:].zfill(digits)


def get_token():
    headers = {
        "Accept": "application/json",
        "Accept-Language": "en-US,en;q=0.9",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    }

    s = requests.Session()
    s.headers.update(headers)

    data1 = f'{{"fy_id":"{base64.b64encode(f"{username}".encode()).decode()}","app_id":"2"}}'
    r1 = s.post("https://api-t2.fyers.in/vagator/v2/send_login_otp_v2", data=data1)

    request_key = r1.json()["request_key"]
    data2 = f'{{"request_key":"{request_key}","otp":{totp(totp_key)}}}'
    r2 = s.post("https://api-t2.fyers.in/vagator/v2/verify_otp", data=data2)
    assert r2.status_code == 200, f"Error in r2:\n {r2.text}"

    request_key = r2.json()["request_key"]
    data3 = f'{{"request_key":"{request_key}","identity_type":"pin","identifier":"{base64.b64encode(f"{pin}".encode()).decode()}"}}'
    r3 = s.post("https://api-t2.fyers.in/vagator/v2/verify_pin_v2", data=data3)
    assert r3.status_code == 200, f"Error in r3:\n {r3.json()}"

    headers = {"authorization": f"Bearer {r3.json()['data']['access_token']}", "content-type": "application/json; charset=UTF-8"}
    data4 = f'{{"fyers_id":"{username}","app_id":"{client_id[:-4]}","redirect_uri":"{redirect_uri}","appType":"100","code_challenge":"","state":"abcdefg","scope":"","nonce":"","response_type":"code","create_cookie":true}}'
    r4 = s.post("https://api.fyers.in/api/v2/token", headers=headers, data=data4)
    assert r4.status_code == 308, f"Error in r4:\n {r4.json()}"

    parsed = urlparse(r4.json()["Url"])
    auth_code = parse_qs(parsed.query)["auth_code"][0]

    session = fyersModel.SessionModel(client_id=client_id, secret_key=secret_key, redirect_uri=redirect_uri, response_type="code", grant_type="authorization_code")
    session.set_token(auth_code)
    response = session.generate_token()
    return response["access_token"]


def get_profile(token):
    fyers = fyersModel.FyersModel(client_id=client_id, token=token, log_path=os.getcwd())
    return fyers.get_profile()


def main():
    token = read_file()
    if token is None:
        token = get_token()

    resp = get_profile(token)

    if "error" in resp["s"] or "error" in resp["message"] or "expired" in resp["message"]:
        token = get_token()
        resp = get_profile(token)

    write_file(token)
    print("Fyers access token is saved in `access_token.txt` file.")
    print(resp)


if __name__ == "__main__":
    main()

In [4]:
import pandas as pd
from fyers_apiv3.FyersWebsocket import data_ws
with open('access_token.txt', 'r') as file:
    tk = file.read().strip()

print(tk)
# df = pd.DataFrame()
# df = resp
# tk = df['access_token']
# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=tk, log_path="")

#variables
buypos = 0
sellpos = 0
stoploss = 0
entry = 0
bflag = 0
sflag = 0
spos = 0
bpos = 0
exit = 0
target = 0
cstrike = ''
pstrike = ''
row = -2
expiry = 'NSE:BANKNIFTY24522'
sym = 'NSE:NIFTYBANK-INDEX'
gain = 0
fmflag = 0
fimflag = 0
emadata5 = pd.DataFrame()
emadata15 = pd.DataFrame()
date_from = '2024-05-21'
date_to = '2024-05-22'
#get data function
def getdata(sym, res, rfrom, rto):
    global emadata5, emadata15
    cdata = {
            "symbol":sym,
            "resolution":str(res),
            "date_format":"1",
            "range_from":rfrom,
            "range_to":rto,
            "cont_flag":"0"
            }
    response = fyers.history(data=cdata)
    data = pd.DataFrame.from_dict(response['candles'])
    cols = ['datetime', 'open', 'high','low','close', 'volume']
    data.columns = cols
    data['datetime'] = pd.to_datetime(data['datetime'], unit="s")
    data['datetime'] = data['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    data['datetime'] = data['datetime'].dt.tz_localize(None)
    data = data.set_index('datetime')
    data['ema'] = data['close'].ewm(span = 5 , min_periods=5).mean()
    if (res == 5):
        emadata5 = data
    if (res == 15):
        emadata15 = data
# on message function
getdata(sym, 5, '2024-05-21','2024-05-22')
emadata5
getdata(sym, 15, '2024-05-21','2024-05-22')
emadata15 
def onmessage(message):
    #symb = message['symbol']
    #ltp = message['ltp']
    #print(message)
    t = time.localtime()
    cmin = time.strftime("%M", t)
    csec = time.strftime("%S", t)
    global buypos, sellpos,  stoploss, exit, pstrike,cstrike,  target, bflag , sflag, sym, fmflag, fimflag, bpos, spos, gain, entry
    if (int(cmin) % 5 == 0  and int(csec) >=1 and  fmflag == 0):
        print("5 ema data updated")
        getdata(sym,5, '2024-05-21','2024-05-22')
        fmflag = 1
        if (spos == 0):
            sflag = 0
    if (int(cmin) % 5 != 0 and fmflag == 1):
        fmflag = 0
    if (int(cmin) % 15 == 0  and int(csec) >=1 and fimflag == 0):
        print("15 ema data updated")
        getdata(sym,15, '2024-05-21','2024-05-22')
        fimflag = 1
        if (bpos == 0):
            bflag = 0
    if (int(cmin) % 15 != 0 and fimflag == 1):
        fimflag = 0
#     ema = emadata['ema'].iloc[-2]
#     l =  emadata['low'].iloc[-2]
    print(f"{message}")
    if (emadata5['close'].iloc[row] > emadata5['ema'].iloc[row] 
        and emadata5['high'].iloc[row] > emadata5['ema'].iloc[row]
        and emadata5['open'].iloc[row] > emadata5['ema'].iloc[row]
        and emadata5['low'].iloc[row] > emadata5['ema'].iloc[row]
        and message['ltp'] < emadata5['low'].iloc[row]):
        ltp = message['ltp'] 
        sp = int(round(ltp,-2))
        if (spos==0 and sflag==0):
            spos = sflag = 1
            entry = message['ltp']
            stoploss = emadata5['high'].iloc[row]
            target = message['ltp'] - ((emadata5['high'].iloc[row] - emadata5['low'].iloc[row])*3)
            print('sell entry')
            pstrike = expiry + str(sp) + "PE"
            # data = {
            #     "symbol": str(pstrike),
            #     "qty":15,
            #     "type":2,
            #     "side":1,
            #     "productType":"MARGIN",
            #     "limitPrice":0,
            #     "stopPrice":0,
            #     "validity":"DAY",
            #     "disclosedQty":0,
            #     "offlineOrder":False,
            #     }
            # print(f"entry {pstrike}")
            # response = fyers.place_order(data=data)
    if (emadata15['close'].iloc[row] < emadata15['ema'].iloc[row] 
        and emadata15['high'].iloc[row] < emadata15['ema'].iloc[row]
        and emadata15['open'].iloc[row] < emadata15['ema'].iloc[row]
        and emadata15['low'].iloc[row] < emadata15['ema'].iloc[row]
        and message['ltp'] > emadata15['high'].iloc[row]):
        ltp = message['ltp']
        sp = int(round(ltp,-2))
        if (bpos==0 and bflag==0):
            bpos = bflag = 1
            entry = message['ltp']
            stoploss = emadata15['low'].iloc[row]
            target = message['ltp'] + ((emadata15['high'].iloc[row] - emadata15['low'].iloc[row])*3)
            print('buy entry')
            cstrike = expiry + str(sp) + "CE"
            # data = {
            #     "symbol": str(cstrike),
            #     "qty":15,
            #     "type":2,
            #     "side":1,
            #     "productType":"MARGIN",
            #     "limitPrice":0,
            #     "stopPrice":0,
            #     "validity":"DAY",
            #     "disclosedQty":0,
            #     "offlineOrder":False,
            #     }
            # print(f"entry {cstrike}")
            # response = fyers.place_order(data=data)
    if (spos==1 and message['ltp'] > stoploss):
            gain += entry - stoploss
            spos = 0
            stoploss = 0
            entry = 0
            target = 0
            # data = {
            #     "symbol": str(pstrike),
            #     "qty":15,
            #     "type":2,
            #     "side":-1,
            #     "productType":"MARGIN",
            #     "limitPrice":0,
            #     "stopPrice":0,
            #     "validity":"DAY",
            #     "disclosedQty":0,
            #     "offlineOrder":False,
            #     }
            # print(f"entry {pstrike}")
            # response = fyers.place_order(data=data)
    if (spos==1 and message['ltp'] <= target):
            gain += entry - target 
            spos = 0
            stoploss = 0
            entry = 0
            target = 0
            # data = {
            #     "symbol": str(pstrike),
            #     "qty":15,
            #     "type":2,
            #     "side":-1,
            #     "productType":"MARGIN",
            #     "limitPrice":0,
            #     "stopPrice":0,
            #     "validity":"DAY",
            #     "disclosedQty":0,
            #     "offlineOrder":False,
            #     }
            # print(f"entry {pstrike}")
            # response = fyers.place_order(data=data)
    if (bpos==1 and message['ltp'] < stoploss):
            gain +=  stoploss - entry
            bpos = 0
            stoploss = 0
            entry = 0
            target = 0
            # data = {
            #     "symbol": str(cstrike),
            #     "qty":15,
            #     "type":2,
            #     "side":-1,
            #     "productType":"MARGIN",
            #     "limitPrice":0,
            #     "stopPrice":0,
            #     "validity":"DAY",
            #     "disclosedQty":0,
            #     "offlineOrder":False,
            #     }
            # print(f"entry {cstrike}")
            # response = fyers.place_order(data=data)
    if (bpos==1 and message['ltp'] >= target):
            gain +=  target - entry 
            bpos = 0
            stoploss = 0
            entry = 0
            target = 0
            # data = {
            #     "symbol": str(cstrike),
            #     "qty":15,
            #     "type":2,
            #     "side":-1,
            #     "productType":"MARGIN",
            #     "limitPrice":0,
            #     "stopPrice":0,
            #     "validity":"DAY",
            #     "disclosedQty":0,
            #     "offlineOrder":False,
            #     }
            # print(f"entry {cstrike}")
            # response = fyers.place_order(data=data)
def onerror(message):
    print("Error:", message)
def onclose(message):
    print("Connection closed:", message)
def onopen():
    data_type = "SymbolUpdate"
    symbols = [sym]
    fyersdata.subscribe(symbols=symbols, data_type=data_type)
    fyersdata.keep_running()
# Replace the sample access token with your actual access token obtained from Fyers
access_token = client_id + ":" + tk
# Create a FyersDataSocket instance with the provided parameters
fyersdata = data_ws.FyersDataSocket(
    access_token=access_token,       # Access token in the format "appid:accesstoken"
    log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
    litemode=True,                  # Lite mode disabled. Set to True if you want a lite response.
    write_to_file=False,              # Save response in a log file instead of printing it.
    reconnect=True,                  # Enable auto-reconnection to WebSocket on disconnection.
    on_connect=onopen,               # Callback function to subscribe to data upon connection.
    on_close=onclose,                # Callback function to handle WebSocket connection close events.
    on_error=onerror,                # Callback function to handle WebSocket errors.
    on_message=onmessage             # Callback function to handle incoming messages from the WebSocket.
)
# Establish a connection to the Fyers WebSocket
fyersdata.connect()

eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MjMyNzE0ODUsImV4cCI6MTcyMzMzNjIyNSwibmJmIjoxNzIzMjcxNDg1LCJhdWQiOlsieDowIiwieDoxIiwiZDoxIiwiZDoyIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbXR3azl5dDFsTGhFWUszcDY0VlhOY3ppeDN2VXdDTUJjd3N0RlZ6Zm1nMU5RY0hqT0tPa2pac2lTd3FCMkc5NVFzOHVhQ3h6azNSZ2g3S1l1S2hkYjh0alhYRWdyZGpjYVFlb0tiT1FNbEtsR0w5WT0iLCJkaXNwbGF5X25hbWUiOiJTQU5KRUVWICBCSEFSR0FWQSIsIm9tcyI6IksxIiwiaHNtX2tleSI6IjJkZGY1MjhhM2I0MTNhYTA0NTJhZDY4NWVlNDk0YTJjYzVjNTAyMGEyZDkzMDZmY2RkNTllMmRmIiwiZnlfaWQiOiJYUzgyMTY4IiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.O9Ob2BvhqSzMjrt57srlhSZYy3RNWOKpXuV8RffEE4c
{'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'}
Error: KeyError: The key 'ltp' is missing in the response.
{'type': 'lit', 'code': 200, 'message': 'Lite Mode On', 's': 'ok'}
Error: KeyError: The key 'ltp' is missing in the response.


{'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'}
Error: KeyError: The key 'ltp' is missing in the response.
{'ltp': 50484.5, 'symbol': 'NSE:NIFTYBANK-INDEX', 'type': 'if'}


In [5]:











print(f"buypos = {buypos}")
print(f"sellpos = {sellpos}")
print(f"stoploss = {stoploss}")
print(f"entry = {entry}")
print(f"bflag = {bflag}")
print(f"sflag = {sflag}")
print(f"spos = {spos}")
print(f"bpos = {bpos}")
print(f"exit = {exit}")
print(f"target = {target}")
print(f"cstrike = '{cstrike}'")
print(f"pstrike = '{pstrike}'")
print(f"gain = {gain}")

buypos = 0
sellpos = 0
stoploss = 0
entry = 0
bflag = 0
sflag = 0
spos = 0
bpos = 0
exit = 0
target = 0
cstrike = ''
pstrike = ''
gain = 0


In [6]:
print(f"buypos = {buypos}")
print(f"sellpos = {sellpos}")
print(f"stoploss = {stoploss}")
print(f"entry = {entry}")
print(f"bflag = {bflag}")
print(f"sflag = {sflag}")
print(f"spos = {spos}")
print(f"bpos = {bpos}")
print(f"exit = {exit}")
print(f"target = {target}")
print(f"cstrike = '{cstrike}'")
print(f"pstrike = '{pstrike}'")
print(f"gain = {gain}")

buypos = 0
sellpos = 0
stoploss = 0
entry = 0
bflag = 0
sflag = 0
spos = 0
bpos = 0
exit = 0
target = 0
cstrike = ''
pstrike = ''
gain = 0
